In [1]:
import os,sys
os.chdir('C:/Users/arpit.goel/Documents/Projects/Kaggle/13.TrafficPrediction')
from datetime import datetime
from datetime import date
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import xgboost as xgb
from sklearn.grid_search import GridSearchCV   #Perforing grid search

import matplotlib.pyplot as plt # to visualize data
from pandas.tools.plotting import autocorrelation_plot # to visualize and configure the parameters of ARIMA model
from statsmodels.tsa.arima_model import ARIMA # to make an ARIMA model that fits the data

df_train=pd.read_csv('train_aWnotuB.csv',parse_dates=['DateTime'])
df_test=pd.read_csv('test_BdBKkAj.csv',parse_dates=['DateTime'])

monthly_regression_params={1: [2.93, 18.59], 2: [0.9, 6.18], 3: [0.5, 8.65], 4: [0.17, 5.19]}

C:\Users\arpit.goel\AppData\Local\Continuum2\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\arpit.goel\AppData\Local\Continuum2\Anaconda2\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
C:\Users\arpit.goel\AppData\Local\Continuum2\Anaconda2\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Pleas

In [2]:
def get_vars(df1):
    df=df1.copy()
    df['year']=df['DateTime'].dt.year
    df['month']=df['DateTime'].dt.month
    df['day']=df['DateTime'].dt.day
    df['hour']=df['DateTime'].dt.hour
    df['day_of_week']=df['DateTime'].dt.dayofweek
    df['week_nbr']=df['DateTime'].dt.week
    df['flag_sunday']=(df['day_of_week']==6).astype(np.int64)
    df['flag_saturday']=(df['day_of_week']==5).astype(np.int64)
    df['flag_friday']=(df['day_of_week']==4).astype(np.int64)
    df['flag_monday']=(df['day_of_week']==0).astype(np.int64)
    df['flag_weekday']=(df['day_of_week']<=4).astype(np.int64)
    df['flag_junction_1']=(df['Junction']==1).astype(np.int64)
    df['flag_junction_2']=(df['Junction']==2).astype(np.int64)
    df['flag_junction_3']=(df['Junction']==3).astype(np.int64)
    df['flag_junction_4']=(df['Junction']==4).astype(np.int64)
    df['flag_last_day_of_month']=(df['day']==df['month'].map({1:31,2:28,3:31,4:30,5:31,6:30,7:31,8:31,9:30,10:31,11:30,12:31})).astype(np.int64)
    df['encoded_month']=12*df['year']+df['month']-24191
    df['date']=10000*df['year']+100*df['month']+df['day']
    df['encoded_date']=(df['DateTime']-date(2015,11,1)).dt.days
    slope=df['Junction'].map(lambda x: monthly_regression_params[x][0])
    intercept=df['Junction'].map(lambda x: monthly_regression_params[x][1])
    df['monthly_avg']=np.round(intercept+slope*df['encoded_month'])
    return df

train=get_vars(df_train)
train_oct=train[train['month']==10]
train=pd.concat([train,train_oct,train_oct])
train=train.sort_values(by='DateTime')


In [49]:
from sklearn.decomposition import PCA
X=train.groupby(['DateTime','Junction'])['Vehicles'].max().unstack().fillna(0)
pca = PCA(n_components=2)
pca.fit(X)
print(pca.explained_variance_ratio_)  
comp0=pca.transform(X)[:,0];comp1=pca.transform(X)[:,1]
X['Comp1']=comp1;X['Comp0']=comp0
X['month']=X.index.month
X['year']=X.index.year
X['day']=X.index.day
X['hour']=X.index.hour
X['day_of_week']=X.index.dayofweek
X['week_nbr']=X.index.week
X['flag_sunday']=(X['day_of_week']==6).astype(np.int64)
X['flag_saturday']=(X['day_of_week']==5).astype(np.int64)
X['flag_friday']=(X['day_of_week']==4).astype(np.int64)
X['flag_monday']=(X['day_of_week']==0).astype(np.int64)
X['flag_weekday']=(X['day_of_week']<=4).astype(np.int64)
X['flag_last_day_of_month']=(X['day']==X['month'].map({1:31,2:28,3:31,4:30,5:31,6:30,7:31,8:31,9:30,10:31,11:30,12:31})).astype(np.int64)
X['encoded_month']=12*X['year']+X['month']-24191
X['date']=10000*X['year']+100*X['month']+X['day']
X.columns=[str(x) for x in X.columns]
rf_models={}


[ 0.86524635  0.10543413]


In [50]:
X.groupby(['year','week_nbr'])[['Comp0','Comp1']].mean().to_clipboard()

In [151]:
regression_params={}
regression_params[0]=stats.linregress(X['encoded_month'],X['Comp0'])
regression_params[1]=stats.linregress(X['encoded_month'],X['Comp1'])

ins=X.head(int(0.95*len(X)))
oos=X.tail(len(X)-len(ins))
oot=get_vars(df_test).drop_duplicates(['DateTime'])

get_avgs=lambda x,y: ins.groupby(x)['Comp%d'%y].mean()

def get_other_vars(df):
    for var,level in [['week_hour',['week_nbr','hour']],['month_hour',['month','hour']],\
                     ['weekend_hour',['flag_weekday','hour']],['weekday_hour',['day_of_week','hour']],\
                     ['week',['week_nbr']],['month',['month']],['weekday',['day_of_week']],['hour',['hour']]]:
        avgs1=get_avgs(level,0).reset_index().rename(columns={'Comp0':'avg0_%s'%var})
        avgs2=get_avgs(level,1).reset_index().rename(columns={'Comp1':'avg1_%s'%var})
        df=pd.merge(df,avgs1,on=level,how='left')
        df=pd.merge(df,avgs2,on=level,how='left')
        df['avg_monthly_comp0']=df['encoded_month']*regression_params[0][0]+regression_params[0][1]
        df['avg_monthly_comp1']=df['encoded_month']*regression_params[1][0]+regression_params[1][1]
    return df
ins=get_other_vars(ins)
oos=get_other_vars(oos)
oot=get_other_vars(oot)

print regression_params[0][0],regression_params[0][1]
print regression_params[1][0],regression_params[1][1]
ins[['avg_monthly_comp0','avg_monthly_comp1']].describe().T

2.98304720037 -28.3046041101
-0.206069683604 1.95528948143


,count,mean,std,min,25%,50%,75%,max
avg_monthly_comp0,13862.0,-1.492041,16.317554,-28.304604,-16.372415,-1.457179,13.458057,25.390245
avg_monthly_comp1,13862.0,0.103071,1.127221,-1.753965,-0.929686,0.100662,1.131011,1.955289


In [152]:
print ins.columns
model_cols=[u'year', u'month', u'day',u'hour', u'day_of_week', u'week_nbr']+\
            [x for x in ins.columns if 'flag' in x]+\
            [x for x in ins.columns if 'avg' in x]

#model_cols=['monthly_avg','avg_weekday_hour','week_nbr','avg_week_hour','day','avg_week_timetag','day_of_week','Junction','hour','year']

target='Comp0'
xgtrain = xgb.DMatrix(ins[model_cols], ins[target])
xgvalid = xgb.DMatrix(oos[model_cols], oos[target])
xgtest = xgb.DMatrix(oot[model_cols])
watchlist  = [(xgvalid,'eval'), (xgtrain,'train')]

param = {'max_depth':3, 'eta':0.01, 'silent':1, 'objective':'reg:linear', 'min_child_weight':4,'booster':'gblinear',\
         'nthread':3, 'gamma': 0.0,'subsample':0.75, 'colsample_bytree':0.9, 'learning_rate':0.1,'reg_alpha':0.1}
print param
param['eval_metric'] = 'rmse'
plst = param.items()
evallist = [(xgvalid,'oos'), (xgtrain,'ins')]
model = xgb.train(plst, xgtrain, 200, evallist)
ins_pred_actual  = model.predict(xgtrain)
oos_pred_actual = model.predict(xgvalid)
oot_pred_actual = model.predict(xgtest)


Index([u'1', u'2', u'3', u'4', u'Comp1', u'Comp0', u'month', u'year', u'day',
       u'hour', u'day_of_week', u'week_nbr', u'flag_sunday', u'flag_saturday',
       u'flag_friday', u'flag_monday', u'flag_weekday',
       u'flag_last_day_of_month', u'encoded_month', u'date', u'avg0_week_hour',
       u'avg1_week_hour', u'avg_monthly_comp0', u'avg_monthly_comp1',
       u'avg0_month_hour', u'avg1_month_hour', u'avg0_weekend_hour',
       u'avg1_weekend_hour', u'avg0_weekday_hour', u'avg1_weekday_hour',
       u'avg0_week', u'avg1_week', u'avg0_month', u'avg1_month',
       u'avg0_weekday', u'avg1_weekday', u'avg0_hour', u'avg1_hour'],
      dtype='object')
{'reg_alpha': 0.1, 'learning_rate': 0.1, 'booster': 'gblinear', 'colsample_bytree': 0.9, 'silent': 1, 'nthread': 3, 'min_child_weight': 4, 'subsample': 0.75, 'eta': 0.01, 'objective': 'reg:linear', 'max_depth': 3, 'gamma': 0.0}
[0]	oos-rmse:39.4376	ins-rmse:16.8165
[1]	oos-rmse:36.0619	ins-rmse:13.6518
[2]	oos-rmse:32.8604	ins-rmse:11.9

[187]	oos-rmse:17.5125	ins-rmse:7.64583
[188]	oos-rmse:17.5125	ins-rmse:7.64502
[189]	oos-rmse:17.5121	ins-rmse:7.64421
[190]	oos-rmse:17.5124	ins-rmse:7.64342
[191]	oos-rmse:17.5117	ins-rmse:7.64262
[192]	oos-rmse:17.5118	ins-rmse:7.64183
[193]	oos-rmse:17.512	ins-rmse:7.64104
[194]	oos-rmse:17.5115	ins-rmse:7.64026
[195]	oos-rmse:17.5118	ins-rmse:7.63948
[196]	oos-rmse:17.5109	ins-rmse:7.63871
[197]	oos-rmse:17.5113	ins-rmse:7.63794
[198]	oos-rmse:17.5114	ins-rmse:7.63717
[199]	oos-rmse:17.5115	ins-rmse:7.63641


In [144]:
model_cols=[u'year', u'month', u'day',u'hour', u'day_of_week', u'week_nbr']+\
            [x for x in ins.columns if 'flag' in x]+\
            [x for x in ins.columns if 'avg' in x]

target='Comp0'
regr = RandomForestRegressor(n_estimators=1, max_depth=4, random_state=0,min_samples_leaf=100,verbose=0)
regr.fit(ins[model_cols],ins[target])
feature_importance=pd.Series(regr.feature_importances_,index=model_cols).sort_values(ascending=False)
feature_importance=feature_importance.cumsum()
feature_importance=feature_importance[feature_importance<0.999]
print feature_importance
model_cols=feature_importance.index
regr.fit(ins[model_cols],ins[target])
print np.sqrt(mean_squared_error(regr.predict(ins[model_cols]), ins[target]))
print np.sqrt(mean_squared_error(regr.predict(oos[model_cols]), oos[target]))
rf_models[0]=[model_cols,regr]

avg_monthly_comp1    0.490075
avg0_weekday_hour    0.959257
week_nbr             0.977702
avg_monthly_comp0    0.991027
dtype: float64
7.81242539642
15.2194473559


In [145]:
model_cols=[u'year', u'month', u'day',u'hour', u'day_of_week', u'week_nbr']+\
            [x for x in ins.columns if 'flag' in x]+\
            [x for x in ins.columns if 'avg' in x]

target='Comp1'
regr = RandomForestRegressor(n_estimators=1, max_depth=3, random_state=0,min_samples_leaf=100,verbose=0)
regr.fit(ins[model_cols],ins[target])
feature_importance=pd.Series(regr.feature_importances_,index=model_cols).sort_values(ascending=False)
feature_importance=feature_importance.cumsum()
feature_importance=feature_importance[feature_importance<0.99]
print feature_importance
model_cols=feature_importance.index
regr.fit(ins[model_cols],ins[target])
print np.sqrt(mean_squared_error(regr.predict(ins[model_cols]), ins[target]))
print np.sqrt(mean_squared_error(regr.predict(oos[model_cols]), oos[target]))
rf_models[1]=[model_cols,regr]

avg1_week_hour       0.746214
avg1_weekday_hour    0.917546
avg_monthly_comp0    0.972204
dtype: float64
7.03046915784
8.82730087676


In [146]:
for data in [ins,oos,oot]:
    data['Pred0']=rf_models[0][1].predict(data[rf_models[0][0]])
    data['Pred1']=rf_models[1][1].predict(data[rf_models[1][0]])
    junction_traffic=pca.inverse_transform(data[['Pred0','Pred1']])
    data['P1']=np.round(junction_traffic[:,0])
    data['P2']=np.round(junction_traffic[:,1])
    data['P3']=np.round(junction_traffic[:,2])
    data['P4']=np.round(junction_traffic[:,3])
oot_scored=oot.groupby('DateTime')[['P1','P2','P3','P4']].max().stack().reset_index()
oot_scored=pd.DataFrame(oot_scored,columns=['DateTime','level_1','Vehicles'])
oot_scored['Junction']=oot_scored['level_1'].map(lambda x: int(x[1]))
oot_scored=pd.merge(oot_scored,df_test,on=['Junction','DateTime'])
oot_scored[['ID','Vehicles']].to_csv('PCA_RF.csv',index=False)
print df_test.shape,oot_scored.shape

(11808, 3) (11808, 5)


In [154]:
target=1

model_cols=[u'year', u'month', u'day',u'hour', u'day_of_week', u'week_nbr']+\
            [x for x in ins.columns if 'flag' in x]+\
            [x for x in ins.columns if 'avg' in x]
model_cols=['avg_monthly_comp0']
target='Comp0'

#import statsmodels.api as sm
#ins['constant']=1
#mod = sm.OLS(ins[target],ins[model_cols+['constant']])
#res = mod.fit()
#print(res.summary())
#ins[model_cols+['Comp0','Comp1']].corr().to_clipboard()

from sklearn import linear_model
for alpha0 in [0,0.001,0.01,0.1,1,10,100,1000,10000]:
    reg = linear_model.Ridge(alpha=alpha0)
    reg.fit(ins[model_cols],ins[target]) 
    print alpha0
    print np.sqrt(mean_squared_error(reg.predict(ins[model_cols]), ins[target]))
    print np.sqrt(mean_squared_error(reg.predict(oos[model_cols]), oos[target]))
    #print pd.Series(reg.coef_,index=model_cols).sort_values()


0
16.9157479897
30.1368094237
0.001
16.9157479897
30.1368094247
0.01
16.9157479897
30.1368094333
0.1
16.9157479897
30.1368095191
1
16.9157479897
30.1368103771
10
16.9157479897
30.136818957
100
16.9157479953
30.1369047634
1000
16.9157485472
30.1377635152
10000
16.9158034757
30.1464193863
